In [1]:
import numpy as np
import tensorflow as tf
import joblib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod

# Suppress warnings
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
print("Libraries imported.")

/Users/mahima/Desktop/ENC/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/mahima/Desktop/ENC/env/lib/python3.12/site-packages/art/estimators/certification/__init__.py:30: UserWarning: PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality
  warnings.warn("PyTorch not found. Not importing DeepZ or Interval Bound Propagation functionality")


Libraries imported.


In [ ]:
print("Loading model and data...")

model = load_model('ids_model.h5')
print("Model loaded.")

X_test = np.load('X_test_ids.npy')
y_test_ohe = np.load('y_test_ids_ohe.npy')
y_test_int = np.load('y_test_ids_int.npy')

label_encoder = joblib.load('label_encoder_ids.joblib')
class_names = label_encoder.classes_

print(f"Data loaded. X_test shape: {X_test.shape}")
print(f"Classes: {class_names}")

print("\nChecking baseline accuracy...")
y_pred_clean = model.predict(X_test, verbose=0)
y_pred_clean_int = np.argmax(y_pred_clean, axis=1)

clean_acc = accuracy_score(y_test_int, y_pred_clean_int)
print(f"Baseline (Clean) Accuracy: {clean_acc * 100:.2f}%")

classifier = KerasClassifier(model=model, clip_values=(X_test.min(), X_test.max()))
print("ART Classifier created.")

Loading model and data...
Model loaded.
Data loaded. X_test shape: (210876, 78, 1)
Classes: ['BENIGN' 'Bot' 'DDoS' 'DoS GoldenEye' 'DoS Hulk' 'DoS Slowhttptest'
 'DoS slowloris' 'FTP-Patator' 'Heartbleed' 'Infiltration' 'PortScan'
 'SSH-Patator']

Checking baseline accuracy...
Baseline (Clean) Accuracy: 90.87%
ART Classifier created.


In [ ]:
EPSILON = 0.1

print(f"\nGenerating FGSM attack samples (epsilon={EPSILON})...")
print("This may take a few minutes depending on dataset size...")

attack_fgsm = FastGradientMethod(estimator=classifier, eps=EPSILON)

n_samples = 10000 
X_test_subset = X_test[:n_samples]
y_test_subset_ohe = y_test_ohe[:n_samples]
y_test_subset_int = y_test_int[:n_samples]

X_test_adv_fgsm = attack_fgsm.generate(x=X_test_subset)
print("Adversarial samples generated.")


Generating FGSM attack samples (epsilon=0.1)...
This may take a few minutes depending on dataset size...


/Users/mahima/Desktop/ENC/env/lib/python3.12/site-packages/art/estimators/classification/keras.py:305: UserWarning: Loss function is a plain function, not a Keras loss object. Cannot set reduction; assuming per-sample loss.
  warnings.warn(


Adversarial samples generated.


In [ ]:
print("\nEvaluating model on FGSM examples...")

y_pred_adv = model.predict(X_test_adv_fgsm, verbose=0)
y_pred_adv_int = np.argmax(y_pred_adv, axis=1)

adv_acc = accuracy_score(y_test_subset_int, y_pred_adv_int)

print("\n" + "="*40)
print(f"RESULTS: FGSM Attack (eps={EPSILON})")
print("="*40)
print(f"Clean Accuracy (on subset): {accuracy_score(y_test_subset_int, np.argmax(model.predict(X_test_subset, verbose=0), axis=1)) * 100:.2f}%")
print(f"Attack Accuracy:            {adv_acc * 100:.2f}%")
print(f"Accuracy Drop:              {(clean_acc - adv_acc) * 100:.2f}%")
print("="*40)

print("\nClassification Report (Adversarial):")
print(classification_report(y_test_subset_int, y_pred_adv_int, target_names=class_names, zero_division=0))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test_subset_int, y_pred_adv_int))

print("\nSaving adversarial samples...")
np.save('X_adv_fgsm_ids.npy', X_test_adv_fgsm)
np.save('y_test_fgsm_ids_int.npy', y_test_subset_int)
np.save('y_test_fgsm_ids_ohe.npy', y_test_subset_ohe)
print("✓ Saved: X_adv_fgsm_ids.npy")
print("✓ Saved: y_test_fgsm_ids_int.npy")
print("✓ Saved: y_test_fgsm_ids_ohe.npy")

print("\n✓ FGSM attack completed successfully!")


Evaluating model on FGSM examples...

RESULTS: FGSM Attack (eps=0.1)
Clean Accuracy (on subset): 90.64%
Attack Accuracy:            62.29%
Accuracy Drop:              28.58%

Classification Report (Adversarial):
                  precision    recall  f1-score   support

          BENIGN       0.65      0.69      0.67      4769
             Bot       0.01      0.19      0.01        16
            DDoS       0.76      0.66      0.70      1128
   DoS GoldenEye       0.12      0.71      0.20        85
        DoS Hulk       0.92      0.62      0.74      2207
DoS Slowhttptest       0.30      0.71      0.42        51
   DoS slowloris       0.22      0.59      0.31        58
     FTP-Patator       0.00      0.00      0.00        76
      Heartbleed       0.00      0.00      0.00         0
    Infiltration       0.00      0.00      0.00         0
        PortScan       0.95      0.44      0.60      1553
     SSH-Patator       0.04      0.11      0.05        57

        accuracy               